In [99]:
import json
import os
from glob import glob
from PIL import Image
import re
import requests
from tqdm import *
from datetime import datetime
import dateutil
import pytz

In [106]:
API_KEY = 'xxx'
API_KEY_SECRET = 'xxx'
BEARER_TOKEN = 'xxx'

In [63]:
headers = {'Authorization': f'Bearer {BEARER_TOKEN}'}

In [66]:
requests.get('https://api.twitter.com/2/users/by/username/EichlerMaor', headers=headers).json()

{'data': {'id': '4004052706',
  'name': 'maor eichler',
  'username': 'EichlerMaor'}}

In [33]:
records = json.load(open('/home/noam/Downloads/maor.json'))
records.sort(key=lambda d: d['created_at'])

In [34]:
# mogrify -path ../small-screenshots/ -resize '50%' -define png:compression-filter=5 -define png:compression-level=9 -define png:compression-strategy=1 -define png:exclude-chunk=all *.png

In [55]:
img_fns = {int(os.path.basename(fn)[:-4]): fn for fn in glob('/home/noam/Downloads/maor/screenshots/*.png')}

In [57]:
with open('/home/noam/Downloads/maor/tweets.html', 'w') as f:
    f.write('<!DOCTYPE html>')
    f.write('<html>\n')
    f.write('<head><meta charset="utf-8"/></head>')
    f.write('<body>\n')
    for d in records:
        tid = int(d['id'])
        assert tid in img_fns
        if tid in img_fns:
            fn = img_fns[tid]
            img = Image.open(fn)
            width, height = img.size
            f.write(f'<img loading="lazy" src="screenshots/{tid}.png" width="{width}" height="{height}">\n<br>\n')
            f.write(f'<a href="https://twitter.com/EichlerMaor/status/{tid}">link</a>\n<br>\n')
            f.write(d['text'])
            f.write('<br><br><br>\n\n\n')

In [43]:
len(records)

3161

In [58]:
all_tweets_txt = open('/home/noam/Downloads/maor/EichlerMaor.txt').read()

In [61]:
all_ids = re.findall(r'EichlerMaor/status/(\d+)', all_tweets_txt)

In [77]:
requests.get('https://api.twitter.com/2/tweets', headers=headers, params={
    'ids': ','.join(all_ids[:3]),
    'expansions': 'referenced_tweets.id,referenced_tweets.id.author_id',
    'tweet.fields': 'created_at,author_id',
    'user.fields': 'username',
}).json()

{'data': [{'text': '@Ovadyah @RealJStreets @FleurHassanN @DanielSeidemann It’s not the holy place. It’s the parking lot for the holy place. Where transportation vehicles are normally found.',
   'id': '1502022296034291719',
   'created_at': '2022-03-10T20:43:16.000Z',
   'referenced_tweets': [{'type': 'replied_to', 'id': '1502020716547526662'}],
   'author_id': '4004052706'},
  {'text': '@sapienist @FleurHassanN @DanielSeidemann It is in Jerusalem. The questioner is a Jerusalemite. It is hardly unreasonable for her to suspect that he is referring to her and everyone line her.',
   'id': '1502021514408964105',
   'created_at': '2022-03-10T20:40:09.000Z',
   'referenced_tweets': [{'type': 'replied_to', 'id': '1502018779416465419'}],
   'author_id': '4004052706'},
  {'text': '@GeraldNGOM @Haaretz @levy_haaretz The shift to English destroyed it',
   'id': '1501953831864832003',
   'created_at': '2022-03-10T16:11:13.000Z',
   'referenced_tweets': [{'type': 'replied_to', 'id': '1501816283997

In [80]:
responses = []
for i in tqdm(range(0, len(all_ids), 100)):
    cur_ids = all_ids[i:i+100]
    r = requests.get('https://api.twitter.com/2/tweets', headers=headers, params={
        'ids': ','.join(cur_ids),
        'expansions': 'referenced_tweets.id,referenced_tweets.id.author_id',
        'tweet.fields': 'created_at,author_id',
        'user.fields': 'username',
    })
    r.raise_for_status()
    responses.append(r)

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 121/121 [02:59<00:00,  1.48s/it]


In [82]:
tweets = {}
users = {}
for r in responses:
    r = r.json()
    for tweet in r['data']:
        tweets[tweet['id']] = tweet
    for tweet in r['includes']['tweets']:
        tweets[tweet['id']] = tweet
    for user in r['includes']['users']:
        users[user['id']] = user

In [105]:
os.chdir('/home/noam/Downloads/maor')
tz = pytz.timezone('Asia/Jerusalem')
with open('tweets.html', 'w') as f:
    f.write('<!DOCTYPE html>')
    f.write('<html lang="en">\n')
    f.write('<head>\n')
    f.write("<title>Maor Eichler's Tweets</title>\n")
    f.write('<meta charset="utf-8"/>\n')
    f.write('<link rel="stylesheet" href="https://cdn.jsdelivr.net/npm/bootstrap@3.4.1/dist/css/bootstrap.min.css" integrity="sha384-HSMxcRTRxnN+Bdg0JdbxYKrThecOKuH5zCYotlSAcp1+c8xmyTe9GYg1l9a69psu" crossorigin="anonymous">')
    f.write('</head>\n')
    f.write('<body style="width: 550px; padding: 10px;">\n')
    for tid in all_ids:
        img_fn = f'shots/{tid}.webp'
        img = Image.open(img_fn)
        width, height = img.size
        url = f"https://twitter.com/EichlerMaor/status/{tid}"
        tweet = tweets[tid]
        created_at = dateutil.parser.parse(tweet['created_at']).astimezone(tz).strftime('%d/%m/%Y %H:%M')
        f.write(f'<img loading="lazy" src="{img_fn}" width="{width}" height="{height}">\n<br>\n')
        f.write(f'<a href="{url}">{url}</a>\n<br>\n')
        f.write(f'{created_at}<br>\n')
        f.write(tweet['text'])
        f.write('<br><br><br>\n\n\n')
    f.write('</body>\n')
    f.write('</html>\n')

In [84]:
Image.open('/home/noam/Downloads/maor/shots-webp/1502022296034291719.webp').size

(550, 350)

In [91]:
tweets['1502022296034291719']

{'text': '@Ovadyah @RealJStreets @FleurHassanN @DanielSeidemann It’s not the holy place. It’s the parking lot for the holy place. Where transportation vehicles are normally found.',
 'id': '1502022296034291719',
 'created_at': '2022-03-10T20:43:16.000Z',
 'referenced_tweets': [{'type': 'replied_to', 'id': '1502020716547526662'}],
 'author_id': '4004052706'}

In [103]:
dateutil.parser.parse('2022-03-10T20:43:16.000Z').astimezone(pytz.timezone('Asia/Jerusalem')).strftime('%d/%m/%Y %H:%M')

'10/03/2022 22:43'

In [100]:
pytz.timezone('Asia/Jerusalem')

<DstTzInfo 'Asia/Jerusalem' LMT+2:21:00 STD>